In [ ]:
import requests
import time
import geopandas as gpd
from shapely.geometry import Point
import csv
import json

# List of cities
cities = [
    "New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio",
    "San Diego", "Dallas", "Austin", "Jacksonville", "San Jose", "Indianapolis",
    "San Francisco", "Columbus", "Fort Worth", "Charlotte", "Detroit", "Oklahoma City", "El Paso"
]

# Base URL of the Overpass API
overpass_url = "http://overpass-api.de/api/interpreter"

# Function to build the Overpass query for neighborhoods
def build_neighbourhood_query(city_name):
    return f"""
    [out:json];
    area["name"="{city_name}"]->.a;
    (
      way["place"~"neighbourhood|suburb|locality"](area.a);
      relation["place"~"neighbourhood|suburb|locality"](area.a);
    );
    out center;
    """

# Generic function to get centroids
def get_centroids(city, query_builder):
    query = query_builder(city)
    try:
        response = requests.post(overpass_url, data={"data": query})
        response.raise_for_status()
        data = response.json()

        centroids = []
        if "elements" in data and len(data["elements"]) > 0:
            for element in data["elements"]:
                if "center" in element:
                    centroids.append({
                        "city": city,
                        "name": element.get("tags", {}).get("name", "Unknown"),
                        "latitude": element["center"]["lat"],
                        "longitude": element["center"]["lon"]
                    })
            print(f"{city}: {len(centroids)} centroids found")
        else:
            print(f"{city}: No center found")
        return centroids
    except Exception as e:
        print(f"Error for {city}: {e}")
        return []

# Function to get the centroids of all cities and neighborhoods
def get_all_centroids(cities, query_builder):
    all_centroids = []
    for city in cities:
        print(f"Checking for {city}...")
        centroids = get_centroids(city, query_builder)
        all_centroids.extend(centroids)
        time.sleep(1)  # Delay to avoid rate limit
    return all_centroids

# Save data to CSV
def save_to_csv(data, output_file):
    if data:
        keys = data[0].keys()
        with open(output_file, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(data)
        print(f"Data saved in {output_file}")
    else:
        print("No data to save")

# Save data to JSON
def save_to_json(data, output_file):
    if data:
        with open(output_file, mode="w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Data saved in {output_file}")
    else:
        print("No data to save")

# Run for neighborhoods
neighbourhood_centroids = get_all_centroids(cities, build_neighbourhood_query)
save_to_csv(neighbourhood_centroids, "neighbourhood_centroids.csv")
save_to_json(neighbourhood_centroids, "neighbourhood_centroids.json")


# Function to get the city center using Nominatim with a User-Agent header
def get_city_center_nominatim(city_name):
    nominatim_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": city_name,
        "format": "json",
        "limit": 1
    }
    headers = {
        "User-Agent": "HomeXplore/1.0 (carlovstan@gmail.com)"
    }
    try:
        response = requests.get(nominatim_url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()

        if data:
            return {
                "city": city_name,
                "latitude": float(data[0]["lat"]),
                "longitude": float(data[0]["lon"])
            }
        else:
            print(f"{city_name}: No data found with Nominatim")
            return None
    except Exception as e:
        print(f"Error for {city_name}: {e}")
        return None


# city_centers = []
# for city in cities:
#     print(f"Fetching center for {city}...")
#     center = get_city_center_nominatim(city)
#     if center:
#         city_centers.append(center)
#     time.sleep(1)  # Delay to avoid rate limit

# Run for cities
# save_to_csv(city_centers, "city_centers.csv")
# save_to_json(city_centers, "city_centers.json")

Verifica per New York...
New York: 150 centroidi trovati
Verifica per Los Angeles...
Los Angeles: 51 centroidi trovati
Verifica per Chicago...
Chicago: 30 centroidi trovati
Verifica per Houston...
Houston: 154 centroidi trovati
Verifica per Phoenix...
Phoenix: 19 centroidi trovati
Verifica per Philadelphia...
Philadelphia: 4 centroidi trovati
Verifica per San Antonio...
San Antonio: 143 centroidi trovati
Verifica per San Diego...
San Diego: 23 centroidi trovati
Verifica per Dallas...
Dallas: 24 centroidi trovati
Verifica per Austin...
Austin: 11 centroidi trovati
Verifica per Jacksonville...
Jacksonville: 37 centroidi trovati
Verifica per San Jose...
San Jose: 30 centroidi trovati
Verifica per Indianapolis...
Indianapolis: 12 centroidi trovati
Verifica per San Francisco...
San Francisco: 43 centroidi trovati
Verifica per Columbus...
Columbus: 116 centroidi trovati
Verifica per Fort Worth...
Fort Worth: 3 centroidi trovati
Verifica per Charlotte...
Charlotte: 100 centroidi trovati
Verif